In [1]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [125]:
import pandas as pd 
import numpy as np
from scipy import stats

In [191]:
df_train = pd.read_csv('Downloads/ds3-which-whisky/train.csv')
df_train.head()

,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.5,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.9,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.0,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.5,1.0
4,6,Davin de Kergommeaux,"After 40 years in barrels, the trademark Canad...",199.0,96,45.0,NaN


In [192]:
df_train.shape

(2874, 7)

In [193]:
df_train['category'].isna().sum()

288

In [194]:
df_train = df_train.dropna()

In [195]:
df_test = pd.read_csv('Downloads/ds3-which-whisky/test.csv')
df_test.head()

,id,author,description,price,ratingValue,pert_alcohol
0,955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [196]:
df_test.shape

(288, 6)

In [197]:
df_train['category'].value_counts(normalize=True)

1.0    0.621163
2.0    0.179725
3.0    0.120759
4.0    0.078352
Name: category, dtype: float64

In [198]:
df_train.describe()

,id,price,ratingValue,pert_alcohol,category
count,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000
mean,2098.447900,230.675081,86.274637,47.984917,1.656300
std,1184.492955,1038.712769,4.560668,6.332968,0.968287
min,1.000000,7.500000,60.000000,33.000000,1.000000
25%,1097.750000,50.000000,83.000000,43.000000,1.000000
50%,2148.500000,80.000000,86.000000,46.000000,1.000000
75%,3139.250000,140.000000,89.000000,51.200000,2.000000
max,4157.000000,26650.000000,97.000000,98.600000,4.000000


In [199]:
df_test.describe()

,id,price,ratingValue,pert_alcohol
count,288.000000,279.000000,288.000000,284.000000
mean,2027.979167,194.132616,86.586806,48.542606
std,1096.101447,549.077153,4.011234,6.102620
min,6.000000,17.000000,73.000000,40.000000
25%,1154.000000,50.000000,84.000000,45.000000
50%,2068.500000,80.000000,87.000000,46.150000
75%,2869.000000,140.000000,89.000000,51.800000
max,4122.000000,7000.000000,96.000000,72.050000


In [200]:
# NLP Libraries
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

In [201]:
# Import Statements
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer


In [202]:
from sklearn.model_selection import train_test_split

X = df_train[['description']]
y = df_train[['category']]

X_train, X_cross_val, y_train, y_cross_val = train_test_split(
    X, y, test_size=len(df_test), random_state=42)

X_train.shape, X_cross_val.shape, y_train.shape, y_cross_val.shape

((2188, 1), (288, 1), (2188, 1), (288, 1))

In [203]:
# Create Pipeline

vect = TfidfVectorizer(stop_words='english')
sgdc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

In [204]:
# Fit Pipeline
pipe.fit(df_train['description'], df_train['category'])

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [226]:
# Experiment Management
from sklearn.model_selection import GridSearchCV

parameters = {
    'vect__max_df': (0.5, 0.75, 1),
    'clf__max_iter': (5, 10),
    'clf__class_weight': (None, "balanced"),
}

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1)

grid_search.fit(df_train['description'], df_train['category'])

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    5.7s finished
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1), 'clf__max_iter': (5, 10), 'clf__class_weight': (None, 'balanced')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [227]:
best = grid_search.best_estimator_
best

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [228]:
grid_search.best_score_

0.9293214862681745

In [220]:
X_test = df_test['description']

In [209]:
X_test.shape

(288,)

In [171]:
y_pred_submission = pd.DataFrame(best.predict(X_test))
y_pred_submission = y_pred_submission.astype(int)

In [172]:
y_pred_submission.head()

,0
0,2
1,2
2,4
3,1
4,1


In [173]:
y_pred_submission[0].value_counts().sum()

288

In [174]:
y_pred_submission[0].value_counts(normalize=True)

1    0.649306
2    0.173611
3    0.107639
4    0.069444
Name: 0, dtype: float64

In [65]:
sample_submission = pd.read_csv('Downloads/ds3-which-whisky/whisky_sample_submission.csv')
submission = sample_submission.copy()
submission['category'] = y_pred_submission
submission.to_csv('NLP_1', index=False)

In [ ]:
# files = ['mega1.csv', 'mega2.csv', 'mega3.csv', 'mega4.csv',
#          'mega5.csv', 'mega6.csv', 'mega7.csv', 'mega8.csv',
#          'mega9.csv', 'mega10.csv']

# submissions = (pd.read_csv(file)[['category']] for file in files)
# ensemble = pd.concat(submissions, axis='columns')
# majority_vote = ensemble.mode(axis='columns')[0]

# sample_submission = pd.read_csv('sample_submission.csv')
# submission = sample_submission.copy()
# submission['status_group'] = majority_vote
# submission.to_csv('mega-model.csv', index=False)